In [2]:
#IMPORTING scikit packages
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [11]:
#IMPORTING SETUP PACKAGES
import pickle
import os.path
import os
import pandas as pd
import numpy as np

import psycopg2
import timeit
from datetime import datetime, timedelta, date
import datetime as dt
from calendar import monthrange
import random
import io
import boto3
import pyarrow as pa
import pyarrow.parquet as pq

from s3fs.core import S3FileSystem

import s3fs


%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)

In [12]:
df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top_seller_YTD_group_by.pickle')

df

df_no_group = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top_seller_YTD.pickle')

In [13]:
print(df.dtypes)
print(df_no_group.dtypes)

min_date                  object
max_date                  object
item_code                 object
item_price               float64
delivery_weeks             int64
avg(sales)               float64
bin                        int32
item_parent_item_code     object
item_main_category        object
item_sub_category_1       object
item_sub_category_2       object
log_price                float64
log_delivery_weeks       float64
log_sales                float64
dtype: object
item_code                 object
update_date               object
item_price               float64
delivery_weeks             int64
sales                      int64
item_parent_item_code     object
item_main_category        object
item_sub_category_1       object
item_sub_category_2       object
log_price                float64
log_delivery_weeks       float64
log_sales                float64
dtype: object


In [20]:
X = df['log_price']
X = sm.add_constant(X)
wls_model_no_fe = sm.WLS(exog=X,endog=df['log_sales'], weights =df['bin'] )
res_wls_no_fe = wls_model_no_fe.fit()
print(res_wls_no_fe.summary())

                            WLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.018
Model:                            WLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3.705
Date:                Wed, 09 Dec 2020   Prob (F-statistic):             0.0557
Time:                        15:20:06   Log-Likelihood:                -242.57
No. Observations:                 200   AIC:                             489.1
Df Residuals:                     198   BIC:                             495.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.1408      0.396      7.922      0.0

In [21]:
reg_model_no_fe = smf.ols("log_sales ~ log_price",df_no_group).fit()
print(reg_model_no_fe.summary())


                            OLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     32.66
Date:                Wed, 09 Dec 2020   Prob (F-statistic):           1.20e-08
Time:                        15:20:10   Log-Likelihood:                -3606.1
No. Observations:                3230   AIC:                             7216.
Df Residuals:                    3228   BIC:                             7228.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2323      0.176     18.360      0.0

In [23]:
wls_model_with_fe = smf.wls("log_sales ~ log_price + C(item_code)", df, weights=df['bin']).fit()#+ log_delivery_weeks
print(wls_model_with_fe.summary())


                            WLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.700
Model:                            WLS   Adj. R-squared:                  0.684
Method:                 Least Squares   F-statistic:                     44.06
Date:                Wed, 09 Dec 2020   Prob (F-statistic):           3.69e-44
Time:                        15:22:33   Log-Likelihood:                -124.08
No. Observations:                 200   AIC:                             270.2
Df Residuals:                     189   BIC:                             306.4
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [24]:
reg_model_with_fe = smf.ols("log_sales ~ log_price + C(item_code)", df_no_group).fit()
print(reg_model_with_fe.summary())



                            OLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                     118.2
Date:                Wed, 09 Dec 2020   Prob (F-statistic):          3.26e-210
Time:                        15:22:50   Log-Likelihood:                -3117.2
No. Observations:                3230   AIC:                             6256.
Df Residuals:                    3219   BIC:                             6323.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [25]:
wls_model_with_fe_w_del = smf.wls("log_sales ~ log_price + log_delivery_weeks + C(item_code)", df, weights=df['bin']).fit()#+ log_delivery_weeks
print(wls_model_with_fe_w_del.summary())

                            WLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.747
Model:                            WLS   Adj. R-squared:                  0.733
Method:                 Least Squares   F-statistic:                     50.55
Date:                Wed, 09 Dec 2020   Prob (F-statistic):           3.10e-50
Time:                        15:27:25   Log-Likelihood:                -106.85
No. Observations:                 200   AIC:                             237.7
Df Residuals:                     188   BIC:                             277.3
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [26]:
reg_model_with_fe_w_del = smf.ols("log_sales ~ log_price + + log_delivery_weeks + C(item_code)", df_no_group).fit()
print(reg_model_with_fe_w_del.summary())


                            OLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.279
Method:                 Least Squares   F-statistic:                     114.6
Date:                Wed, 09 Dec 2020   Prob (F-statistic):          2.02e-221
Time:                        15:27:33   Log-Likelihood:                -3088.6
No. Observations:                3230   AIC:                             6201.
Df Residuals:                    3218   BIC:                             6274.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte